In [1]:
import os 


import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import * 
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.compose import *
from sklearn.impute import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.decomposition import *
from sklearn.feature_selection import *


In [2]:
pwd

'/home/xavier/DataScience/P4/P4_repo'

In [3]:
#cd ..

/tmp/P4_repo


/tmp/P4_repo/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [42]:
files = os.listdir()
files = [f for f in files if f.endswith(".csv")]
#files = [f for f in files if "cleaned" in f]
filename = files[0]
filename

'seattle_2016.csv'

In [43]:
df = pd.read_csv(filename)

In [38]:
df["SiteEnergyUse(kBtu)"].isna().sum()

5

In [44]:
data=df.drop(df.loc[df['SiteEnergyUseWN(kBtu)'].isna()].index).reset_index(drop=True)
data.shape

(3370, 46)

In [45]:
col_to_keep=['Latitude','Longitude','YearBuilt','NumberofBuildings','NumberofFloors','PropertyGFAParking',
             'PropertyGFABuilding(s)','ENERGYSTARScore']

In [46]:
y_name = "SiteEnergyUse(kBtu)"
y = data[y_name]
X = data[col_to_keep]

X


,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFAParking,PropertyGFABuilding(s),ENERGYSTARScore
0,47.61220,-122.33799,1927,1.0,12,0,88434,60.0
1,47.61317,-122.33393,1996,1.0,11,15064,88502,61.0
2,47.61393,-122.33810,1969,1.0,41,196718,759392,43.0
3,47.61412,-122.33664,1926,1.0,10,0,61320,56.0
4,47.61375,-122.34047,1980,1.0,18,62000,113580,75.0
...,...,...,...,...,...,...,...,...
3365,47.56722,-122.31154,1990,1.0,1,0,12294,46.0
3366,47.59625,-122.32283,2004,1.0,1,0,16000,NaN
3367,47.63644,-122.35784,1974,1.0,1,0,13157,NaN
3368,47.52832,-122.32431,1989,1.0,1,0,14101,NaN


In [47]:
y 

0       7.226362e+06
1       8.387933e+06
2       7.258702e+07
3       6.794584e+06
4       1.417261e+07
            ...     
3365    8.497457e+05
3366    9.502762e+05
3367    5.765898e+06
3368    7.194712e+05
3369    1.152896e+06
Name: SiteEnergyUse(kBtu), Length: 3370, dtype: float64

In [48]:
imputer = KNNImputer(n_neighbors=5)
imputer.fit(X)
X_imputed = imputer.transform(X)
X_imputed = pd.DataFrame(X_imputed, columns=X.columns)
X_imputed

,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFAParking,PropertyGFABuilding(s),ENERGYSTARScore
0,47.61220,-122.33799,1927.0,1.0,12.0,0.0,88434.0,60.0
1,47.61317,-122.33393,1996.0,1.0,11.0,15064.0,88502.0,61.0
2,47.61393,-122.33810,1969.0,1.0,41.0,196718.0,759392.0,43.0
3,47.61412,-122.33664,1926.0,1.0,10.0,0.0,61320.0,56.0
4,47.61375,-122.34047,1980.0,1.0,18.0,62000.0,113580.0,75.0
...,...,...,...,...,...,...,...,...
3365,47.56722,-122.31154,1990.0,1.0,1.0,0.0,12294.0,46.0
3366,47.59625,-122.32283,2004.0,1.0,1.0,0.0,16000.0,82.6
3367,47.63644,-122.35784,1974.0,1.0,1.0,0.0,13157.0,82.6
3368,47.52832,-122.32431,1989.0,1.0,1.0,0.0,14101.0,82.6


In [49]:
y.isna().sum()

0

In [50]:
X_imputed.isna().sum()

Latitude                  0
Longitude                 0
YearBuilt                 0
NumberofBuildings         0
NumberofFloors            0
PropertyGFAParking        0
PropertyGFABuilding(s)    0
ENERGYSTARScore           0
dtype: int64

In [51]:
pipeline=Pipeline([
    ('scaler',StandardScaler()),
    ('estimator',LinearRegression())
])

In [58]:
grid=GridSearchCV(
    pipeline,
    {},
    cv=10,
    scoring=('r2','neg_mean_absolute_error','neg_mean_squared_error'),
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='r2')


In [53]:
grid

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('estimator', LinearRegression())]),
             n_jobs=-1, param_grid={}, refit='r2', return_train_score=True,
             scoring='r2', verbose=2)

In [59]:
grid.fit(X_imputed,y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('estimator', LinearRegression())]),
             n_jobs=-1, param_grid={}, refit='r2', return_train_score=True,
             scoring=('r2', 'neg_mean_absolute_error',
                      'neg_mean_squared_error'),
             verbose=2)

In [60]:
grid.cv_results_

{'mean_fit_time': array([0.01045642]),
 'std_fit_time': array([0.00749416]),
 'mean_score_time': array([0.01080825]),
 'std_score_time': array([0.01178278]),
 'params': [{}],
 'split0_test_r2': array([0.28701135]),
 'split1_test_r2': array([0.41766416]),
 'split2_test_r2': array([-0.51490402]),
 'split3_test_r2': array([0.26506788]),
 'split4_test_r2': array([0.59049442]),
 'split5_test_r2': array([0.47273114]),
 'split6_test_r2': array([0.39985082]),
 'split7_test_r2': array([-0.53199769]),
 'split8_test_r2': array([0.17709138]),
 'split9_test_r2': array([0.94777455]),
 'mean_test_r2': array([0.2510784]),
 'std_test_r2': array([0.43663497]),
 'rank_test_r2': array([1], dtype=int32),
 'split0_train_r2': array([0.82392408]),
 'split1_train_r2': array([0.72560586]),
 'split2_train_r2': array([0.68949374]),
 'split3_train_r2': array([0.68888671]),
 'split4_train_r2': array([0.6882039]),
 'split5_train_r2': array([0.68882404]),
 'split6_train_r2': array([0.68861454]),
 'split7_train_r2': a

In [61]:
def result(grid, log_target=0,transf_feat=0, features=''):
    ''' Fonction retournant un dataframe res recensant les différents résultats du GridSearchCv passé en fonction des paramétres utilisés'''
    res=pd.DataFrame(grid.cv_results_).round(2)
    cols=[i for i in res.columns if 'split' not in i ]
    res=res.loc[:,cols]
    ###
    
    res['log_target']=log_target
    res['transf_feat']=transf_feat
    res['features']=features
    
    ###
    
    return res.sort_values("rank_test_r2", ascending =True)

In [62]:
res=result(grid)
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,mean_test_r2,std_test_r2,rank_test_r2,mean_train_r2,std_train_r2,...,mean_train_neg_mean_absolute_error,std_train_neg_mean_absolute_error,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error,mean_train_neg_mean_squared_error,std_train_neg_mean_squared_error,log_target,transf_feat,features
0,0.01,0.01,0.01,0.01,{},0.25,0.44,1,0.68,0.1,...,-3435548.14,367457.86,-1.505348e+14,2.681642e+14,1,-1.456895e+14,2.947206e+13,0,0,
